In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import datetime as dt
import matplotlib.pyplot as plot

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
#conn = engine.connect()
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
#Use SQLAlchemy `automap_base()` to reflect your tables into classes and save a reference to those 
#classes called `Station` and `Measurement`.
#Example = Base.classes.example

Station = Base.classes.station
Measurement = Base.classes.measurement

In [ ]:
# Create our session (link) from Python to the DB
#Link Python to the database by creating an SQLAlchemy session
session = Session(engine)

In [ ]:
#inspector for column names
inspector = inspect(engine)
inspector.get_table_names()
columns = inspector.get_columns('Measurement')
for column in columns:
    print(column["name"], column["type"])

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
    
lastdate = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
lastdate

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
print("Last Date: ", lastdate)

# Calculate the date one year from the last date in data set.
query_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print("Query Date: ", query_date)

# Perform a query to retrieve the data and precipitation scores
results = session.query(Measurement.date, Measurement.prcp).filter(func.strftime(Measurement.date) >= query_date).all()
results  

# Save the query results as a Pandas DataFrame and set the index to the date column


results_df = pd.DataFrame(results, columns=['date','prcp'])

results_df.set_index('date')
# Sort the dataframe by date
results_df.sort_values('date')

# Use Pandas Plotting with Matplotlib to plot the data
results_df.plot.bar(x='date', y='prcp')
plot.show()


In [56]:
# Use Pandas to calcualte the summary statistics for the precipitation data
groupby_mean = results_df['prcp'].mean()
groupby_median = results_df['prcp'].median()
groupby_variance = results_df['prcp'].var()
groupby_std = results_df['prcp'].std()
groupby_sem = results_df['prcp'].sem()


print("Mean: ", groupby_mean)
print("Median: ", groupby_median)
print("Variance: ", groupby_variance)
print("STD: ", groupby_std)
print("SEM: ", groupby_sem)


Mean:  0.17727857496288965
Median:  0.02
Variance:  0.21269575849618608
STD:  0.4611895038877035
SEM:  0.01025879279145186


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()